In [8]:
import pandas as pd
import utm
import numpy as np
import random
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import h2o
from h2o.automl import H2OAutoML
from utils.from_latlon import from_latlon
from utils.preprocessing import preprocess
from utils.postprocessing import postprocess_data, location_averaging, calculate_error

seed = 38 # Seed for train/val split

In [9]:
# Specify data paths
train_data = 'SBTF_data/input_data/train_test_data/train_data_combined.xlsx'
test_data = 'SBTF_data/input_data/train_test_data/test_data.xlsx'
radio_tower_xy_path = 'SBTF_data/input_data/radio_tower_locations/RTEastNorth_1group.xlsx'

# Variable parameters
freq = '3min' # Frequency of data

# Fixed parameters
routine = 'training'
dimensions = ['xOffset', 'yOffset']


In [10]:
def preprocess_sim_data(sim_data, freq, tower_locs, routine):
    
    sim_dat_filt, predictors = preprocess(sim_data, freq, routine)
     
    # Calculate easting and northing from lat long
    sim_dat_filt['easting'], sim_dat_filt['northing'], sim_dat_filt['zone_num'], sim_dat_filt['zone_letter'] = from_latlon(sim_dat_filt['POINT_Y'].values, sim_dat_filt['POINT_X'].values)

    # Create a dictionary of the coordinates of the towers
    offset_dict = tower_locs.set_index('TowerID').to_dict()
    point_x = offset_dict['POINT_X']
    point_y = offset_dict['POINT_Y']

    # Standardise the coordinates so that the tower location == 0 on both the x and y axes.
    sim_dat_filt['xOffset'] = sim_dat_filt['easting'] - sim_dat_filt['TowerID'].map(point_x).fillna(0)
    sim_dat_filt['yOffset'] = sim_dat_filt['northing'] - sim_dat_filt['TowerID'].map(point_y).fillna(0)
    
    return sim_dat_filt, predictors


In [11]:
# Get training data
train_data = pd.read_excel(train_data)
train_data['DateAndTime'] = pd.to_datetime(train_data['DateAndTime'])

# Get testing data
test_data = pd.read_excel(test_data)
test_data['DateAndTime'] = pd.to_datetime(test_data['DateAndTime'])

# Get tower locations
tower_locs = pd.read_excel(radio_tower_xy_path)

In [12]:
train_data_preproc, predictors_train = preprocess_sim_data(train_data, freq, tower_locs, routine)


c:\Users\s5236256\Documents\GitHub\ml4rt\utils\preprocessing.py:10: FutureWarning: The provided callable <function std at 0x00000275B3F80A60> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  .agg(['mean', 'count', np.std])


In [13]:
len(train_data_preproc['Point_ID'].unique().tolist())

461

In [7]:
# Preprocess the training and testing data
train_data_preproc, predictors_train = preprocess_sim_data(train_data, freq, tower_locs, routine)
test_data_preproc, predictors_test = preprocess_sim_data(test_data, freq, tower_locs, routine)

unique_ids = train_data_preproc['Point_ID'].unique().tolist()

# Initialise h2o
h2o.init(nthreads = 2)

# For loop to sample training data, train, and test the models and export the result
train_data_length = len(unique_ids)
n = 5
run = 1

while n <= train_data_length:
    # Create a random subset of the training dataframe based on random Point_IDs, with the number of points equal to n
    random.shuffle(unique_ids)
    subset = unique_ids[:n]
    train_data_preproc_sample = train_data_preproc[train_data_preproc['Point_ID'].isin(subset)]

    test_data_preproc_n = test_data_preproc.copy()
    results = []

    # Train, save and test the models for each dimension
    for dimension in dimensions:
        # print(f"Training model for {dimension}")
        # Train the model
        variables = predictors_train + [dimension]
        training_input = train_data_preproc_sample[variables]
        train = h2o.H2OFrame(training_input)
        aml = H2OAutoML(max_models=20, seed=seed, stopping_metric='MAE', sort_metric='MAE', max_runtime_secs=600)
        aml.train(x=predictors_train, y=dimension, training_frame=train)
        print(aml.leaderboard)

        # Save the leader model
        # h2o.save_model(aml.leader, path = f"Example_data\Output\Trained_models\{dimension}", force=True)

        # Make predictions on the test data
        test = h2o.H2OFrame(test_data_preproc_n)
        preds = aml.leader.predict(test)

        # Save predictions to a new column in the test dataframe
        pred_column_name = f"{dimension}_pred"
        test_data_preproc_n[pred_column_name] = preds.as_data_frame()

    # Post process the test predictions to calculate location from the radio tower locations
    test_predictions_tower = postprocess_data(test_data_preproc_n, tower_locs)

    # Location averaging functions
    test_location_estimates = location_averaging(test_predictions_tower)
    test_location_estimates = calculate_error(test_location_estimates)

    # Calculate the mean absolute error of UTM_predictions['distance'] and the standard error
    mean_error = np.mean(test_location_estimates['error_m'])
    std_error = stats.sem(test_location_estimates['error_m'])

    results.append({'r': run, 'n': n, 'mean_error': mean_error, 'std_error': std_error})
    results_df = pd.DataFrame(results)
    results_df.to_csv(f'SBTF_data/paper_results/sample_size_trial/individual_runs/run_{run}_n{n}_error.csv', index=False)

    print(f'Mean error (+/-SE) = {mean_error} (+/- {std_error})')
    
    n = n+1

# Stop h2o
h2o.cluster().shutdown()


Checking whether there is an H2O instance running at http://localhost:54321.

c:\Users\s5236256\Documents\GitHub\ml4rt\utils\preprocessing.py:10: FutureWarning: The provided callable <function std at 0x00000275B3F80A60> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  .agg(['mean', 'count', np.std])
c:\Users\s5236256\Documents\GitHub\ml4rt\utils\preprocessing.py:10: FutureWarning: The provided callable <function std at 0x00000275B3F80A60> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  .agg(['mean', 'count', np.std])


.... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM Zulu17.42+19-CA (build 17.0.7+7-LTS, mixed mode, sharing)
  Starting server from C:\Users\s5236256\Documents\GitHub\ml4rt\.venv\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\s5236256\AppData\Local\Temp\tmpohl6ow5p
  JVM stdout: C:\Users\s5236256\AppData\Local\Temp\tmpohl6ow5p\h2o_s5236256_started_from_python.out
  JVM stderr: C:\Users\s5236256\AppData\Local\Temp\tmpohl6ow5p\h2o_s5236256_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,Australia/Brisbane
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,1 month and 8 days
H2O_cluster_name:,H2O_from_python_s5236256_yvrl5q
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.922 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
20:46:29.884: AutoML: XGBoost is not available; skipping it.
20:46:29.977: _nfolds param, nfolds cannot be larger than the number of rows (4).
20:46:29.978: _train param, Dropping bad and constant columns: [ant2_std, ant1_count, ant2_count, ant4_count, ant1_mean, ant4_std, ant2_mean, ant4_mean, ant1_std]
20:46:29.994: _nfolds param, nfolds cannot be larger than the number of rows (4).
20:46:29.994: _train param, Dropping bad and constant columns: [ant2_std, ant1_count, ant2_count, ant4_count, ant1_mean, ant4_std, ant2_mean, ant4_mean, ant1_std]
20:46:29.995: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 4.0.
20:46:29.999: _nfolds param, nfolds cannot be larger than the number of rows (4).
20:46:29.999: _train param, Dropping bad and constant columns: [ant2_std, ant1_count, ant2_count, ant4

OSError: Job with key $03017f00000132d4ffffffff$_abececde805a180d85e5a83b258446b3 failed with an exception: water.exceptions.H2OAutoMLException: Aborting AutoML after too many consecutive model failures
stacktrace: 
water.exceptions.H2OAutoMLException: Aborting AutoML after too many consecutive model failures
	at ai.h2o.automl.AutoML.learn(AutoML.java:776)
	at ai.h2o.automl.AutoML.run(AutoML.java:494)
	at ai.h2o.automl.H2OJob$1.compute2(H2OJob.java:33)
	at water.H2O$H2OCountedCompleter.compute(H2O.java:1689)
	at jsr166y.CountedCompleter.exec(CountedCompleter.java:468)
	at jsr166y.ForkJoinTask.doExec(ForkJoinTask.java:263)
	at jsr166y.ForkJoinPool$WorkQueue.runTask(ForkJoinPool.java:976)
	at jsr166y.ForkJoinPool.runWorker(ForkJoinPool.java:1479)
	at jsr166y.ForkJoinWorkerThread.run(ForkJoinWorkerThread.java:104)


In [38]:
test_predictions_tower.to_excel("test_tower_predictions.xlsx", index=False)
test_location_estimates.to_excel("UTM_predictions.xlsx", index=False)

In [39]:
# sorted_df = test_location_estimates.sort_values(by='error_m', ascending=False)
# sorted_df

In [40]:
train_data_preproc.to_excel("train_data_preproc.xlsx", index=False)